In [53]:
import pandas as pd
import numpy as np
from datetime import timedelta
from glob import glob
import matplotlib.pyplot as plt


#pip install plotly
import plotly.express as px

pd.options.plotting.backend='plotly'

import plotly.io as pio
pio.renderers.default="notebook_connected"

import warnings
warnings.filterwarnings('ignore')

import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [54]:
energy= pd.read_csv("ulsan_energy_time.csv")
fcst=pd.read_csv("ulsan_fcst_linear.csv")

In [55]:
energy

,date,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2018-03-01,0.0,4.0,35.0,71.0,82.0,334.0,372.0,346.0,318.0,258.0,160.0,30.0,0.0
1,2018-03-02,0.0,18.0,102.0,218.0,264.0,278.0,362.0,352.0,319.0,263.0,163.0,30.0,0.0
2,2018-03-03,0.0,14.0,75.0,103.0,167.0,143.0,168.0,256.0,311.0,250.0,151.0,31.0,0.0
3,2018-03-04,0.0,20.0,91.0,166.0,257.0,264.0,200.0,105.0,86.0,43.0,20.0,1.0,0.0
4,2018-03-05,0.0,0.0,0.0,3.0,0.0,8.0,14.0,21.0,22.0,21.0,16.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,2021-01-27,0.0,1.0,46.0,148.0,240.0,290.0,306.0,303.0,269.0,199.0,72.0,6.0,0.0
1064,2021-01-28,0.0,0.0,17.0,75.0,160.0,249.0,294.0,294.0,256.0,130.0,77.0,8.0,0.0
1065,2021-01-29,0.0,1.0,56.0,160.0,250.0,310.0,331.0,327.0,289.0,218.0,91.0,6.0,0.0
1066,2021-01-30,0.0,1.0,44.0,144.0,228.0,238.0,292.0,297.0,231.0,140.0,50.0,7.0,0.0


In [56]:
fcst

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 15:00:00,8.0,20.0,14.0,298.0,2.0
1,2018-03-01 16:00:00,7.0,20.0,10.8,298.0,2.0
2,2018-03-01 17:00:00,5.0,20.0,7.5,298.0,2.0
3,2018-03-01 18:00:00,4.0,20.0,4.3,298.0,2.0
4,2018-03-01 19:00:00,4.0,23.0,3.6,303.0,2.0
...,...,...,...,...,...,...
26309,2021-03-03 12:00:00,9.0,45.0,3.1,83.0,3.0
26310,2021-03-03 15:00:00,9.0,45.0,3.0,111.0,3.0
26311,2021-03-03 18:00:00,8.0,55.0,2.2,122.0,3.0
26312,2021-03-03 21:00:00,6.0,65.0,0.9,131.0,3.0


In [57]:
# 시간(hour) 변수 생성
fcst['hour']= fcst['Forecast_time'].str.split(' ').str[1].str.split(':').str[0].astype(int)
fcst['date']= fcst['Forecast_time'].str.split(' ').str[0]

# year montj day추가
year_fcst = fcst['date'].str.split('-').str[0].astype(int)
month_fcst = fcst['date'].str.split('-').str[1].astype(int)
day_fcst = fcst['date'].str.split('-').str[2].astype(int)
fcst['day_int']=year_fcst*10000+month_fcst*100+day_fcst
fcst

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,day_int
0,2018-03-01 15:00:00,8.0,20.0,14.0,298.0,2.0,15,2018-03-01,20180301
1,2018-03-01 16:00:00,7.0,20.0,10.8,298.0,2.0,16,2018-03-01,20180301
2,2018-03-01 17:00:00,5.0,20.0,7.5,298.0,2.0,17,2018-03-01,20180301
3,2018-03-01 18:00:00,4.0,20.0,4.3,298.0,2.0,18,2018-03-01,20180301
4,2018-03-01 19:00:00,4.0,23.0,3.6,303.0,2.0,19,2018-03-01,20180301
...,...,...,...,...,...,...,...,...,...
26309,2021-03-03 12:00:00,9.0,45.0,3.1,83.0,3.0,12,2021-03-03,20210303
26310,2021-03-03 15:00:00,9.0,45.0,3.0,111.0,3.0,15,2021-03-03,20210303
26311,2021-03-03 18:00:00,8.0,55.0,2.2,122.0,3.0,18,2021-03-03,20210303
26312,2021-03-03 21:00:00,6.0,65.0,0.9,131.0,3.0,21,2021-03-03,20210303


In [58]:
# year montj day추가
year_energy = energy['date'].str.split('-').str[0].astype(int)
month_energy = energy['date'].str.split('-').str[1].astype(int)
day_energy = energy['date'].str.split('-').str[2].astype(int)
energy['day_int']=year_energy*10000+month_energy*100+day_energy
energy

,date,7,8,9,10,11,12,13,14,15,16,17,18,19,day_int
0,2018-03-01,0.0,4.0,35.0,71.0,82.0,334.0,372.0,346.0,318.0,258.0,160.0,30.0,0.0,20180301
1,2018-03-02,0.0,18.0,102.0,218.0,264.0,278.0,362.0,352.0,319.0,263.0,163.0,30.0,0.0,20180302
2,2018-03-03,0.0,14.0,75.0,103.0,167.0,143.0,168.0,256.0,311.0,250.0,151.0,31.0,0.0,20180303
3,2018-03-04,0.0,20.0,91.0,166.0,257.0,264.0,200.0,105.0,86.0,43.0,20.0,1.0,0.0,20180304
4,2018-03-05,0.0,0.0,0.0,3.0,0.0,8.0,14.0,21.0,22.0,21.0,16.0,9.0,0.0,20180305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,2021-01-27,0.0,1.0,46.0,148.0,240.0,290.0,306.0,303.0,269.0,199.0,72.0,6.0,0.0,20210127
1064,2021-01-28,0.0,0.0,17.0,75.0,160.0,249.0,294.0,294.0,256.0,130.0,77.0,8.0,0.0,20210128
1065,2021-01-29,0.0,1.0,56.0,160.0,250.0,310.0,331.0,327.0,289.0,218.0,91.0,6.0,0.0,20210129
1066,2021-01-30,0.0,1.0,44.0,144.0,228.0,238.0,292.0,297.0,231.0,140.0,50.0,7.0,0.0,20210130


## 데이터가 온전하게  존재하는 2018-03-02부터 2021-01-31까지의 데이터만 사용한다.

In [59]:
energy=energy[energy['day_int']>=20180302]
fcst=fcst[fcst['day_int']>=20180302]
fcst=fcst[fcst['day_int']<=20210131]

In [60]:
energy


,date,7,8,9,10,11,12,13,14,15,16,17,18,19,day_int
1,2018-03-02,0.0,18.0,102.0,218.0,264.0,278.0,362.0,352.0,319.0,263.0,163.0,30.0,0.0,20180302
2,2018-03-03,0.0,14.0,75.0,103.0,167.0,143.0,168.0,256.0,311.0,250.0,151.0,31.0,0.0,20180303
3,2018-03-04,0.0,20.0,91.0,166.0,257.0,264.0,200.0,105.0,86.0,43.0,20.0,1.0,0.0,20180304
4,2018-03-05,0.0,0.0,0.0,3.0,0.0,8.0,14.0,21.0,22.0,21.0,16.0,9.0,0.0,20180305
5,2018-03-06,0.0,3.0,38.0,110.0,152.0,156.0,287.0,371.0,333.0,278.0,175.0,36.0,0.0,20180306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,2021-01-27,0.0,1.0,46.0,148.0,240.0,290.0,306.0,303.0,269.0,199.0,72.0,6.0,0.0,20210127
1064,2021-01-28,0.0,0.0,17.0,75.0,160.0,249.0,294.0,294.0,256.0,130.0,77.0,8.0,0.0,20210128
1065,2021-01-29,0.0,1.0,56.0,160.0,250.0,310.0,331.0,327.0,289.0,218.0,91.0,6.0,0.0,20210129
1066,2021-01-30,0.0,1.0,44.0,144.0,228.0,238.0,292.0,297.0,231.0,140.0,50.0,7.0,0.0,20210130


In [61]:
fcst
fcst

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,day_int
9,2018-03-02 00:00:00,0.0,40.0,1.3,321.0,2.0,0,2018-03-02,20180302
10,2018-03-02 01:00:00,-0.0,42.0,1.5,314.0,2.0,1,2018-03-02,20180302
11,2018-03-02 02:00:00,-1.0,43.0,1.6,306.0,2.0,2,2018-03-02,20180302
12,2018-03-02 03:00:00,-1.0,45.0,1.8,299.0,2.0,3,2018-03-02,20180302
13,2018-03-02 04:00:00,-1.0,45.0,1.8,301.0,2.0,4,2018-03-02,20180302
...,...,...,...,...,...,...,...,...,...
25612,2021-01-31 19:00:00,9.0,53.0,2.8,195.0,3.0,19,2021-01-31,20210131
25613,2021-01-31 20:00:00,9.0,57.0,3.2,200.0,4.0,20,2021-01-31,20210131
25614,2021-01-31 21:00:00,9.0,60.0,3.5,204.0,4.0,21,2021-01-31,20210131
25615,2021-01-31 22:00:00,10.0,62.0,3.9,201.0,4.0,22,2021-01-31,20210131


## 시간대별 데이터 형성

In [98]:
h=7
ulsan_7=fcst[fcst['hour']==h]
ulsan_7=ulsan_7.drop(['day_int','Forecast_time'],axis=1)
ulsan_7['year'] = ulsan_7['date'].str.split('-').str[0].astype(int)
ulsan_7['month'] = ulsan_7['date'].str.split('-').str[1].astype(int)
ulsan_7['day']= ulsan_7['date'].str.split('-').str[2].astype(int)

ulsan_7['enrgy']=list(energy.iloc[:,h-6])

ulsan_7


,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
16,-1.0,43.0,1.8,308.0,1.0,7,2018-03-02,2018,3,2,0.0
40,4.0,68.0,1.3,292.0,1.0,7,2018-03-03,2018,3,3,0.0
64,9.0,87.0,1.6,316.0,3.0,7,2018-03-04,2018,3,4,0.0
88,7.0,85.0,5.9,30.0,4.0,7,2018-03-05,2018,3,5,0.0
112,4.0,63.0,4.1,21.0,3.0,7,2018-03-06,2018,3,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...
25504,3.0,57.0,5.3,319.0,3.0,7,2021-01-27,2021,1,27,0.0
25528,3.0,68.0,1.5,302.0,3.0,7,2021-01-28,2021,1,28,0.0
25552,-5.0,38.0,10.2,295.0,1.0,7,2021-01-29,2021,1,29,0.0
25576,-1.0,45.0,4.1,242.0,1.0,7,2021-01-30,2021,1,30,0.0


In [99]:
h=8
ulsan_8=fcst[fcst['hour']==h]
ulsan_8=ulsan_8.drop(['day_int','Forecast_time'],axis=1)
ulsan_8['year'] = ulsan_8['date'].str.split('-').str[0].astype(int)
ulsan_8['month'] = ulsan_8['date'].str.split('-').str[1].astype(int)
ulsan_8['day']= ulsan_8['date'].str.split('-').str[2].astype(int)

ulsan_8['enrgy']=list(energy.iloc[:,h-6])

ulsan_8

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
17,-0.0,42.0,1.7,311.0,1.0,8,2018-03-02,2018,3,2,18.0
41,5.0,62.0,1.2,283.0,2.0,8,2018-03-03,2018,3,3,14.0
65,10.0,83.0,1.5,323.0,3.0,8,2018-03-04,2018,3,4,20.0
89,6.0,85.0,7.9,31.0,4.0,8,2018-03-05,2018,3,5,0.0
113,5.0,62.0,3.3,26.0,3.0,8,2018-03-06,2018,3,6,3.0
...,...,...,...,...,...,...,...,...,...,...,...
25505,4.0,53.0,5.2,321.0,3.0,8,2021-01-27,2021,1,27,1.0
25529,5.0,67.0,2.0,267.0,3.0,8,2021-01-28,2021,1,28,0.0
25553,-5.0,37.0,10.2,297.0,1.0,8,2021-01-29,2021,1,29,1.0
25577,0.0,45.0,4.7,242.0,1.0,8,2021-01-30,2021,1,30,1.0


In [100]:
h=9
ulsan_9=fcst[fcst['hour']==h]
ulsan_9=ulsan_9.drop(['day_int','Forecast_time'],axis=1)
ulsan_9['year'] = ulsan_9['date'].str.split('-').str[0].astype(int)
ulsan_9['month'] = ulsan_9['date'].str.split('-').str[1].astype(int)
ulsan_9['day']= ulsan_9['date'].str.split('-').str[2].astype(int)

ulsan_9['enrgy']=list(energy.iloc[:,h-6])

ulsan_9

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
18,0.0,40.0,1.7,315.0,1.0,9,2018-03-02,2018,3,2,102.0
42,6.0,55.0,1.1,275.0,2.0,9,2018-03-03,2018,3,3,75.0
66,11.0,80.0,1.3,331.0,3.0,9,2018-03-04,2018,3,4,91.0
90,6.0,85.0,10.0,32.0,4.0,9,2018-03-05,2018,3,5,0.0
114,6.0,60.0,2.4,31.0,3.0,9,2018-03-06,2018,3,6,38.0
...,...,...,...,...,...,...,...,...,...,...,...
25506,4.0,50.0,5.2,324.0,3.0,9,2021-01-27,2021,1,27,46.0
25530,6.0,65.0,2.4,231.0,3.0,9,2021-01-28,2021,1,28,17.0
25554,-5.0,35.0,10.2,298.0,1.0,9,2021-01-29,2021,1,29,56.0
25578,2.0,45.0,5.2,242.0,1.0,9,2021-01-30,2021,1,30,44.0


In [101]:
h=10
ulsan_10=fcst[fcst['hour']==h]
ulsan_10=ulsan_10.drop(['day_int','Forecast_time'],axis=1)
ulsan_10['year'] = ulsan_10['date'].str.split('-').str[0].astype(int)
ulsan_10['month'] = ulsan_10['date'].str.split('-').str[1].astype(int)
ulsan_10['day']= ulsan_10['date'].str.split('-').str[2].astype(int)

ulsan_10['enrgy']=list(energy.iloc[:,h-6])

ulsan_10

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
19,2.0,37.0,1.5,256.0,1.0,10,2018-03-02,2018,3,2,218.0
43,8.0,50.0,1.4,258.0,2.0,10,2018-03-03,2018,3,3,103.0
67,13.0,77.0,1.4,269.0,3.0,10,2018-03-04,2018,3,4,166.0
91,6.0,87.0,10.7,31.0,4.0,10,2018-03-05,2018,3,5,3.0
115,6.0,60.0,2.9,40.0,3.0,10,2018-03-06,2018,3,6,110.0
...,...,...,...,...,...,...,...,...,...,...,...
25507,6.0,43.0,5.2,335.0,2.0,10,2021-01-27,2021,1,27,148.0
25531,8.0,57.0,4.0,232.0,3.0,10,2021-01-28,2021,1,28,75.0
25555,-3.0,32.0,9.1,299.0,1.0,10,2021-01-29,2021,1,29,160.0
25579,4.0,47.0,5.6,244.0,2.0,10,2021-01-30,2021,1,30,144.0


In [102]:
h=11
ulsan_11=fcst[fcst['hour']==h]
ulsan_11=ulsan_11.drop(['day_int','Forecast_time'],axis=1)
ulsan_11['year'] = ulsan_11['date'].str.split('-').str[0].astype(int)
ulsan_11['month'] = ulsan_11['date'].str.split('-').str[1].astype(int)
ulsan_11['day']= ulsan_11['date'].str.split('-').str[2].astype(int)

ulsan_11['enrgy']=list(energy.iloc[:,h-6])

ulsan_11

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
20,4.0,33.0,1.3,198.0,1.0,11,2018-03-02,2018,3,2,264.0
44,10.0,45.0,1.6,240.0,2.0,11,2018-03-03,2018,3,3,167.0
68,15.0,73.0,1.6,206.0,3.0,11,2018-03-04,2018,3,4,257.0
92,5.0,88.0,11.5,30.0,4.0,11,2018-03-05,2018,3,5,0.0
116,7.0,60.0,3.5,48.0,3.0,11,2018-03-06,2018,3,6,152.0
...,...,...,...,...,...,...,...,...,...,...,...
25508,8.0,37.0,5.3,346.0,2.0,11,2021-01-27,2021,1,27,240.0
25532,9.0,48.0,5.6,234.0,3.0,11,2021-01-28,2021,1,28,160.0
25556,-2.0,28.0,8.0,300.0,1.0,11,2021-01-29,2021,1,29,250.0
25580,6.0,48.0,6.1,247.0,2.0,11,2021-01-30,2021,1,30,228.0


In [104]:
h=12
ulsan_12=fcst[fcst['hour']==h]
ulsan_12=ulsan_12.drop(['day_int','Forecast_time'],axis=1)
ulsan_12['year'] = ulsan_12['date'].str.split('-').str[0].astype(int)
ulsan_12['month'] = ulsan_12['date'].str.split('-').str[1].astype(int)
ulsan_12['day']= ulsan_12['date'].str.split('-').str[2].astype(int)

ulsan_12['enrgy']=list(energy.iloc[:,h-6])

ulsan_12

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
21,6.0,30.0,1.1,139.0,1.0,12,2018-03-02,2018,3,2,278.0
45,12.0,40.0,1.9,223.0,2.0,12,2018-03-03,2018,3,3,143.0
69,17.0,70.0,1.7,144.0,3.0,12,2018-03-04,2018,3,4,264.0
93,5.0,90.0,12.2,29.0,4.0,12,2018-03-05,2018,3,5,8.0
117,7.0,60.0,4.0,57.0,3.0,12,2018-03-06,2018,3,6,156.0
...,...,...,...,...,...,...,...,...,...,...,...
25509,10.0,30.0,5.3,357.0,1.0,12,2021-01-27,2021,1,27,290.0
25533,11.0,40.0,7.2,235.0,3.0,12,2021-01-28,2021,1,28,249.0
25557,0.0,25.0,6.9,301.0,1.0,12,2021-01-29,2021,1,29,310.0
25581,8.0,50.0,6.5,249.0,3.0,12,2021-01-30,2021,1,30,238.0


In [105]:
h=13
ulsan_13=fcst[fcst['hour']==h]
ulsan_13=ulsan_13.drop(['day_int','Forecast_time'],axis=1)
ulsan_13['year'] = ulsan_13['date'].str.split('-').str[0].astype(int)
ulsan_13['month'] = ulsan_13['date'].str.split('-').str[1].astype(int)
ulsan_13['day']= ulsan_13['date'].str.split('-').str[2].astype(int)

ulsan_13['enrgy']=list(energy.iloc[:,h-6])

ulsan_13

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
22,7.0,32.0,1.5,153.0,1.0,13,2018-03-02,2018,3,2,362.0
46,13.0,47.0,2.2,220.0,2.0,13,2018-03-03,2018,3,3,168.0
70,17.0,73.0,2.1,144.0,3.0,13,2018-03-04,2018,3,4,200.0
94,5.0,90.0,12.3,29.0,4.0,13,2018-03-05,2018,3,5,14.0
118,7.0,60.0,4.1,57.0,3.0,13,2018-03-06,2018,3,6,287.0
...,...,...,...,...,...,...,...,...,...,...,...
25510,10.0,32.0,4.9,243.0,1.0,13,2021-01-27,2021,1,27,306.0
25534,11.0,42.0,7.8,244.0,3.0,13,2021-01-28,2021,1,28,294.0
25558,1.0,23.0,6.8,302.0,1.0,13,2021-01-29,2021,1,29,331.0
25582,9.0,48.0,5.7,257.0,3.0,13,2021-01-30,2021,1,30,292.0


In [106]:
h=14
ulsan_14=fcst[fcst['hour']==h]
ulsan_14=ulsan_14.drop(['day_int','Forecast_time'],axis=1)
ulsan_14['year'] = ulsan_14['date'].str.split('-').str[0].astype(int)
ulsan_14['month'] = ulsan_14['date'].str.split('-').str[1].astype(int)
ulsan_14['day']= ulsan_14['date'].str.split('-').str[2].astype(int)

ulsan_14['enrgy']=list(energy.iloc[:,h-6])

ulsan_14

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
23,8.0,33.0,1.9,168.0,1.0,14,2018-03-02,2018,3,2,352.0
47,14.0,53.0,2.6,217.0,3.0,14,2018-03-03,2018,3,3,256.0
71,16.0,77.0,2.5,145.0,4.0,14,2018-03-04,2018,3,4,105.0
95,4.0,90.0,12.5,29.0,4.0,14,2018-03-05,2018,3,5,21.0
119,7.0,60.0,4.1,58.0,3.0,14,2018-03-06,2018,3,6,371.0
...,...,...,...,...,...,...,...,...,...,...,...
25511,11.0,33.0,4.5,130.0,1.0,14,2021-01-27,2021,1,27,303.0
25535,11.0,43.0,8.4,252.0,3.0,14,2021-01-28,2021,1,28,294.0
25559,1.0,22.0,6.6,304.0,1.0,14,2021-01-29,2021,1,29,327.0
25583,10.0,47.0,4.8,265.0,3.0,14,2021-01-30,2021,1,30,297.0


In [107]:
h=15
ulsan_15=fcst[fcst['hour']==h]
ulsan_15=ulsan_15.drop(['day_int','Forecast_time'],axis=1)
ulsan_15['year'] = ulsan_15['date'].str.split('-').str[0].astype(int)
ulsan_15['month'] = ulsan_15['date'].str.split('-').str[1].astype(int)
ulsan_15['day']= ulsan_15['date'].str.split('-').str[2].astype(int)

ulsan_15['enrgy']=list(energy.iloc[:,h-6])

ulsan_15

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
24,9.0,35.0,2.3,182.0,1.0,15,2018-03-02,2018,3,2,319.0
48,15.0,60.0,2.9,214.0,3.0,15,2018-03-03,2018,3,3,311.0
72,16.0,80.0,2.9,145.0,4.0,15,2018-03-04,2018,3,4,86.0
96,4.0,90.0,12.6,29.0,4.0,15,2018-03-05,2018,3,5,22.0
120,7.0,60.0,4.2,58.0,3.0,15,2018-03-06,2018,3,6,333.0
...,...,...,...,...,...,...,...,...,...,...,...
25512,11.0,35.0,4.1,16.0,1.0,15,2021-01-27,2021,1,27,269.0
25536,11.0,45.0,9.0,261.0,3.0,15,2021-01-28,2021,1,28,256.0
25560,2.0,20.0,6.5,305.0,1.0,15,2021-01-29,2021,1,29,289.0
25584,11.0,45.0,4.0,273.0,3.0,15,2021-01-30,2021,1,30,231.0


In [108]:
h=16
ulsan_16=fcst[fcst['hour']==h]
ulsan_16=ulsan_16.drop(['day_int','Forecast_time'],axis=1)
ulsan_16['year'] = ulsan_16['date'].str.split('-').str[0].astype(int)
ulsan_16['month'] = ulsan_16['date'].str.split('-').str[1].astype(int)
ulsan_16['day']= ulsan_16['date'].str.split('-').str[2].astype(int)

ulsan_16['enrgy']=list(energy.iloc[:,h-6])

ulsan_16

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
25,9.0,38.0,2.4,226.0,1.0,16,2018-03-02,2018,3,2,263.0
49,14.0,65.0,3.1,208.0,3.0,16,2018-03-03,2018,3,3,250.0
73,15.0,83.0,2.8,159.0,4.0,16,2018-03-04,2018,3,4,43.0
97,4.0,88.0,12.6,30.0,4.0,16,2018-03-05,2018,3,5,21.0
121,6.0,62.0,4.3,59.0,3.0,16,2018-03-06,2018,3,6,278.0
...,...,...,...,...,...,...,...,...,...,...,...
25513,10.0,40.0,3.8,21.0,1.0,16,2021-01-27,2021,1,27,199.0
25537,8.0,43.0,9.1,272.0,3.0,16,2021-01-28,2021,1,28,130.0
25561,1.0,22.0,5.3,301.0,1.0,16,2021-01-29,2021,1,29,218.0
25585,10.0,50.0,3.2,274.0,2.0,16,2021-01-30,2021,1,30,140.0


In [109]:
h=17
ulsan_17=fcst[fcst['hour']==h]
ulsan_17=ulsan_17.drop(['day_int','Forecast_time'],axis=1)
ulsan_17['year'] = ulsan_17['date'].str.split('-').str[0].astype(int)
ulsan_17['month'] = ulsan_17['date'].str.split('-').str[1].astype(int)
ulsan_17['day']= ulsan_17['date'].str.split('-').str[2].astype(int)

ulsan_17['enrgy']=list(energy.iloc[:,h-6])

ulsan_17

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
26,8.0,42.0,2.4,269.0,1.0,17,2018-03-02,2018,3,2,163.0
50,13.0,70.0,3.3,202.0,3.0,17,2018-03-03,2018,3,3,151.0
74,14.0,87.0,2.6,173.0,4.0,17,2018-03-04,2018,3,4,20.0
98,5.0,87.0,12.5,31.0,4.0,17,2018-03-05,2018,3,5,16.0
122,6.0,63.0,4.3,60.0,3.0,17,2018-03-06,2018,3,6,175.0
...,...,...,...,...,...,...,...,...,...,...,...
25514,8.0,45.0,3.5,25.0,1.0,17,2021-01-27,2021,1,27,72.0
25538,6.0,42.0,9.2,284.0,3.0,17,2021-01-28,2021,1,28,77.0
25562,1.0,23.0,4.2,297.0,1.0,17,2021-01-29,2021,1,29,91.0
25586,8.0,55.0,2.5,276.0,2.0,17,2021-01-30,2021,1,30,50.0


In [111]:
h=18
ulsan_18=fcst[fcst['hour']==h]
ulsan_18=ulsan_18.drop(['day_int','Forecast_time'],axis=1)
ulsan_18['year'] = ulsan_18['date'].str.split('-').str[0].astype(int)
ulsan_18['month'] = ulsan_18['date'].str.split('-').str[1].astype(int)
ulsan_18['day']= ulsan_18['date'].str.split('-').str[2].astype(int)

ulsan_18['enrgy']=list(energy.iloc[:,h-6])

ulsan_18

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
27,8.0,45.0,2.5,313.0,1.0,18,2018-03-02,2018,3,2,30.0
51,12.0,75.0,3.5,196.0,3.0,18,2018-03-03,2018,3,3,31.0
75,13.0,90.0,2.5,187.0,4.0,18,2018-03-04,2018,3,4,1.0
99,5.0,85.0,12.5,32.0,4.0,18,2018-03-05,2018,3,5,9.0
123,5.0,65.0,4.4,61.0,3.0,18,2018-03-06,2018,3,6,36.0
...,...,...,...,...,...,...,...,...,...,...,...
25515,7.0,50.0,3.2,30.0,1.0,18,2021-01-27,2021,1,27,6.0
25539,3.0,40.0,9.3,295.0,3.0,18,2021-01-28,2021,1,28,8.0
25563,0.0,25.0,3.0,293.0,1.0,18,2021-01-29,2021,1,29,6.0
25587,7.0,60.0,1.7,277.0,1.0,18,2021-01-30,2021,1,30,7.0


In [112]:
h=19
ulsan_19=fcst[fcst['hour']==h]
ulsan_19=ulsan_19.drop(['day_int','Forecast_time'],axis=1)
ulsan_19['year'] = ulsan_19['date'].str.split('-').str[0].astype(int)
ulsan_19['month'] = ulsan_19['date'].str.split('-').str[1].astype(int)
ulsan_19['day']= ulsan_19['date'].str.split('-').str[2].astype(int)

ulsan_19['enrgy']=list(energy.iloc[:,h-6])

ulsan_19

,Temperature,Humidity,WindSpeed,WindDirection,Cloud,hour,date,year,month,day,enrgy
28,7.0,50.0,2.5,284.0,1.0,19,2018-03-02,2018,3,2,0.0
52,12.0,75.0,3.1,216.0,3.0,19,2018-03-03,2018,3,3,0.0
76,13.0,90.0,2.4,151.0,4.0,19,2018-03-04,2018,3,4,0.0
100,5.0,82.0,11.6,31.0,4.0,19,2018-03-05,2018,3,5,0.0
124,5.0,65.0,4.0,62.0,3.0,19,2018-03-06,2018,3,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...
25516,7.0,52.0,3.1,27.0,2.0,19,2021-01-27,2021,1,27,0.0
25540,2.0,37.0,8.6,295.0,3.0,19,2021-01-28,2021,1,28,0.0
25564,-1.0,27.0,3.5,291.0,1.0,19,2021-01-29,2021,1,29,0.0
25588,6.0,63.0,2.0,284.0,1.0,19,2021-01-30,2021,1,30,0.0


In [115]:
ulsan_7.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_7.csv",index=False)
ulsan_8.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_8.csv",index=False)
ulsan_9.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_9.csv",index=False)
ulsan_10.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_10.csv",index=False)
ulsan_11.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_11.csv",index=False)
ulsan_12.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_12.csv",index=False)
ulsan_13.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_13.csv",index=False)
ulsan_14.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_14.csv",index=False)
ulsan_15.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_15.csv",index=False)
ulsan_16.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_16.csv",index=False)
ulsan_17.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_17.csv",index=False)
ulsan_18.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_18.csv",index=False)
ulsan_19.to_csv("C:\\Users\\sungb\\파이썬\\태양광 데이콘\\python\\ulsan_19.csv",index=False)